# Creating a Discussion

The simplest task you can accomplish with this library is to create a small discussion between LLMs. 

This guide will teach you the basic setup of the library. You will understand how to setup models, user-agents and how to coordinate them in a discussion. By the end of htis guide, you will be able to run a small discussion with a moderator and save it to the disk for persistence.

## Basics

### The Model

SynDisco can theoretically support any LLM, as long as it is wrapped in a `BaseModel` wrapper. The `BaseModel` class is a very simple interface with one method. This method gives the underlying LLM input, and returns its output to the library.

There already exists a `TransformersModel` class which handles models from the `transformers` python library. In 90% of your applications, this will be enough. We can load a TransformersModel using the following code:

In [1]:
from syndisco.model import TransformersModel

llm = TransformersModel(
    model_path="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    name="test_model",
    max_out_tokens=100,
)

/media/SSD_4TB_2/dtsirmpas/software/miniforge/envs/syndisco/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cuda:1


This will download a small LLM from huggingface. You can substitute the model_path for any similar model in [HuggingFace](https://huggingface.co/) supporting the Transformers library.

### Creating personas

All `actors` can be defined by a `persona`, aka a set of attributes that define them. These attributes can be age, ethnicity, and even include special instructions on how they should behave.

Creating a persona programmatically is simple:

In [2]:
from syndisco.actors import Persona

persona_data = [
    {
        "username": "Emma35",
        "age": 38,
        "sex": "female",
        "education_level": "Bachelor's",
        "sexual_orientation": "Heterosexual",
        "demographic_group": "Latino",
        "current_employment": "Registered Nurse",
        "special_instructions": "",
        "personality_characteristics": [
            "compassionate",
            "patient",
            "diligent",
            "overwhelmed",
        ],
    },
    {
        "username": "Giannis",
        "age": 21,
        "sex": "male",
        "education_level": "College",
        "sexual_orientation": "Pansexual",
        "demographic_group": "White",
        "current_employment": "Game Developer",
        "special_instructions": "",
        "personality_characteristics": [
            "strategic",
            "meticulous",
            "nerdy",
            "hyper-focused",
        ],
    },
]

personas = [Persona(**data) for data in persona_data]

for persona in personas:
    print(persona)

{"username": "Emma35", "age": 38, "sex": "female", "sexual_orientation": "Heterosexual", "demographic_group": "Latino", "current_employment": "Registered Nurse", "education_level": "Bachelor's", "special_instructions": "", "personality_characteristics": ["compassionate", "patient", "diligent", "overwhelmed"]}
{"username": "Giannis", "age": 21, "sex": "male", "sexual_orientation": "Pansexual", "demographic_group": "White", "current_employment": "Game Developer", "education_level": "College", "special_instructions": "", "personality_characteristics": ["strategic", "meticulous", "nerdy", "hyper-focused"]}


Since creating a lot of distinct users is essential in running large-scale experiments, users are usually defined in JSON format. That way, you can change anything without touching your code!

[Here](https://github.com/dimits-ts/synthetic_moderation_experiments/blob/master/data/discussions_input/personas/personas.json) is an applied example of how to mass-define user personas through JSON files.

### Creating the user-agents

Having a `persona` and a `model` we can finally create an `actor`. The actor will personify the selected persona using the model to talk.

Besides a persona and a model, the actors will also need instructions and a context. By convention, all actors share the same context, and all user-agents share the same instructions. Personalized instructions are defined in the actor's persona.

In [3]:
from syndisco.actors import Actor, ActorType


CONTEXT = "You are taking part in an online conversation"
INSTRUCTIONS = "Act like a human would"

actors = [
    Actor(
        model=llm,
        persona=p,
        actor_type=ActorType.USER,
        context=CONTEXT,
        instructions=INSTRUCTIONS
    )
    for p in personas
]

### Managing turn-taking

In real-life discussions, who gets to speak at each point in time is determined by complex social dynamics, which are difficult to realistically model.
However, there are ways with which we can simulate these dynamics. 

SynDisco uses the `TurnManager` class to model turn taking. Two implementations are available by default: Round Robin, and Random Weighted

- `Round Robin` is the simplest, most intuitive way to model turn-taking; everyone gets to talk once per round. Once everyone talks once, they get to talk again in the same sequence

In [4]:
from syndisco.turn_manager import RoundRobin


rrobin_turn_manager = RoundRobin(["John", "Mary", "Howard", "Todd"])

for i in range(10):
    print(next(rrobin_turn_manager))

John
Mary
Howard
Todd
John
Mary
Howard
Todd
John
Mary


- `RandomWeighted` on the other hand throws a weighted coin on each round. If the coin flip succedes, the previous user gets to respond. If not, another user is selected at random

In [5]:
from syndisco.turn_manager import RandomWeighted


rweighted_turn_manager = RandomWeighted(
    names=["John", "Mary", "Howard", "Todd"], p_respond=0.5
)

for i in range(10):
    print(next(rweighted_turn_manager))

Todd
Mary
Howard
John
Howard
Todd
Howard
Mary
Todd
John


## Generating a discussion

Let's start with the most basic task; a single discussion between two user-agents.

Since we only have two users, a RoundRobin approach where each user takes a turn sequentially is sufficient.

Now we can run a simple discussion

In [6]:
from syndisco.jobs import Discussion


turn_manager = RoundRobin([actor.get_name() for actor in actors])
conv = Discussion(next_turn_manager=turn_manager, users=actors)
conv.begin()

 20%|██        | 1/5 [00:02<00:08,  2.05s/it]

User Emma35 posted:
I'm happy to chat with you. How's your day going so far? 



 40%|████      | 2/5 [00:05<00:08,  2.90s/it]

User Giannis posted:
It's going alright, I guess. Been stuck on this game bug for a while
now and I'm trying to figure out why it's not compiling as expected.
But, other than that, I'm good. How about you, how's your day going? 



 60%|██████    | 3/5 [00:10<00:07,  3.80s/it]

User Emma35 posted:
It's been a busy day for me. I've been juggling multiple patients and
trying to keep up with their needs. I love my job as a registered
nurse, but some days are definitely more chaotic than others. I've
been feeling a bit overwhelmed, to be honest. How about you, have you
found any solutions to that game bug you've been stuck on? 



 80%|████████  | 4/5 [00:16<00:04,  4.84s/it]

User Giannis posted:
I totally understand the feeling of being overwhelmed. As a game
developer, I'm used to juggling multiple tasks and deadlines, but it's
not always easy. I've been working on this bug for a while now, and I
think I might have found the issue. It seems like a subtle one, but
it's been causing some inconsistencies in the game's physics engine.
I'm just hoping I can squash it before the next patch.  But, I'm
curious, what kind of patients are you 



100%|██████████| 5/5 [00:23<00:00,  4.68s/it]

User Emma35 posted:
I work in a busy hospital, so I'm constantly surrounded by patients
with all sorts of medical needs. There's the elderly gentleman who's
recovering from a hip replacement surgery, the young mom who's having
trouble with her newborn's reflux, and the patient who's dealing with
a chronic illness that requires constant monitoring. It's a lot to
handle, but I love the feeling of making a difference in their lives.
Speaking of which, I did have a patient who was really grateful for my
care today 



Let's add a moderator to oversee the discussion.

In [7]:
MODERATOR_INSTRUCTIONS = "You are a moderator. Oversee the discussion"

moderator_persona = Persona(
    **{
        "username": "Moderator",
        "age": 41,
        "sex": "male",
        "education_level": "PhD",
        "sexual_orientation": "Pansexual",
        "demographic_group": "White",
        "current_employment": "Moderator",
        "special_instructions": "",
        "personality_characteristics": [
            "strict",
            "neutral",
            "just",
        ],
    }
)

moderator = Actor(
    model=llm,
    persona=moderator_persona,
    actor_type=ActorType.USER,
    context=CONTEXT,
    instructions=MODERATOR_INSTRUCTIONS
)

# remember to update this!
turn_manager = RoundRobin(
    [actor.get_name() for actor in actors] + [moderator.get_name()]
)
conv = Discussion(
    next_turn_manager=turn_manager,
    users=actors + [moderator],
    moderator=moderator,
)
conv.begin()

  0%|          | 0/5 [00:00<?, ?it/s]

User Emma35 posted:
I'm happy to chat with you. How's your day going so far? 



 20%|██        | 1/5 [00:04<00:17,  4.40s/it]

User Moderator posted:
User Emma35, thanks for the friendly greeting. As a moderator, I don't
really have personal experiences or emotions, but I'm functioning
properly and ready to assist with any topics you'd like to discuss.
What's on your mind? 

User Giannis posted:
Thanks for the warm welcome, Emma! I'm glad to be here. As for me,
it's been a pretty typical day of working on some new game mechanics
and debugging code. I'm really excited about the project I'm currently
working on, but it's been a bit of a challenge to get everything just
right. I've been spending most of my free time playing some strategy
games to get my mind warmed up, actually. I'm a bit of a nerd, so I
love that kind 



 40%|████      | 2/5 [00:14<00:23,  7.88s/it]

...

User Moderator posted:
User Giannis, you mentioned that you're working on some new game
mechanics and debugging code. Can you provide more details on the
specific areas that are giving you trouble? Are you experiencing any
issues with balance, gameplay, or perhaps something else? 

User Emma35 posted:
I'd love to hear more about your project, Giannis. As a nurse, I'm
always fascinated by the creative and technical aspects of game
development. I'm sure it's not easy to balance gameplay and mechanics,
especially when trying to make everything just right.  I've always
been impressed by the complexity of game development, and I'm sure
it's not unlike the intricacies of patient care - sometimes you have
to think outside the box to come up with a solution that works for
everyone. What 



 80%|████████  | 4/5 [00:34<00:09,  9.63s/it]

User Moderator posted:
User Giannis, thank you for sharing more about your project. It sounds
like you're facing some interesting challenges in terms of balancing
gameplay and mechanics. Can you tell us a bit more about what specific
areas you're struggling with? For example, are you having trouble with
character movement, puzzle-solving, or perhaps something else?  Also,
I'd like to invite Emma35 to continue the conversation. As a nurse,
her perspective on the complexities of balancing multiple systems and
finding solutions that work for everyone is 

User Giannis posted:
Thanks for having me, Moderator. I'm happy to share more about my
project. To be honest, I'm having a bit of trouble with the balance
between exploration and combat mechanics. I want the player to feel
like they can explore the world without feeling overwhelmed by the
combat system, but at the same time, I want the combat to be
challenging and rewarding.  Specifically, I'm struggling with finding
the right ratio of e

100%|██████████| 5/5 [00:47<00:00,  9.56s/it]

User Moderator posted:
User Moderator posted: User Giannis, that's a great starting point.
Balancing exploration and combat can be a delicate dance. Can you tell
us more about what you've tried so far to achieve this balance? For
example, have you experimented with different combat mechanics, such
as difficulty scaling, enemy AI, or player abilities? Have you also
considered implementing a "rest" or "regeneration" system to give
players a break from combat? We'd also like to hear from Emma35, who 



### Saving a discussion to the disk

It's best practice to save the results of each discussion after it has concluded. This way, no matter what happens to the program executing the discussions, progress will be checkpointed.

The `Discussion` class provides a method for saving its logs and related metadata to a JSON file:

In [8]:
import tempfile
import json

tp = tempfile.NamedTemporaryFile(delete=True)

conv.to_json_file(tp.name)

# if you are running this on Windows, uncomment this line
# tp.close()
with open(tp.name, mode="rb") as f:
    print(json.dumps(json.load(f), indent=2))

{
  "id": "88dc8d0b-8976-45a8-bda1-c15ed7d75450",
  "timestamp": "25-11-17-15-13",
  "users": [
    "Emma35",
    "Giannis",
    "Moderator"
  ],
  "moderator": "Moderator",
  "user_prompts": [
    {
      "context": "You are taking part in an online conversation",
      "instructions": "Act like a human would",
      "type": "1",
      "persona": {
        "username": "Emma35",
        "age": 38,
        "sex": "female",
        "sexual_orientation": "Heterosexual",
        "demographic_group": "Latino",
        "current_employment": "Registered Nurse",
        "education_level": "Bachelor's",
        "special_instructions": "",
        "personality_characteristics": [
          "compassionate",
          "patient",
          "diligent",
          "overwhelmed"
        ]
      }
    },
    {
      "context": "You are taking part in an online conversation",
      "instructions": "Act like a human would",
      "type": "1",
      "persona": {
        "username": "Giannis",
        "age"

Congratulations, you can now run fully synthetic discussions! You may want to experiment with adding more than 2 users or testing more realistic turn taking procedures (for example, check out the `RandomWeighted` turn manager).